In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html

In [2]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.dashboard_objs as dashboard
import IPython.display
from IPython.display import Image

In [3]:
import pandas as pd
import numpy as np
import pickle
from script import *
from sarimax_script import *
from garch_script import *
from appliances_script import *

/usr/local/lib/python3.5/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



In [4]:
plt.rc('figure', figsize=(18.0, 6.0))

In [5]:
pd.set_option('display.max_columns', None) 

In [6]:
df = load_building(545, '2015-5-01 00:00:00', '2016-10-31 00:00:00')
df_weather_austin = load_weather('austin')

In [7]:
df_exog = pickle.load(open('df_exog_545.p','rb'))
arima_exog_results = pickle.load(open('arima_exog_results_545.p', 'rb'))
# df_exog, arima_exog_results = fit_exog_arima(df, df_weather_austin, 545)

In [8]:
mse, rmse, df_exog_train = plot_exog_arima(df, df_exog, arima_exog_results, 545);

In [9]:
start_train, end_train, start_test, end_test = find_dates(545)

In [10]:
df_exog_train

,localhour,Hourly_Usage,Rain,Sleet,Snow,Day_of_Week,Weekend,Temperature,Humidity,Precip_Intensity,forecast
2016-10-01 00:00:00,2016-10-01 00:00:00-05:00,0.842800,0.0,0.0,0.0,5,1,64.67,0.73,0.0000,NaN
2016-10-01 01:00:00,2016-10-01 01:00:00-05:00,0.584133,0.0,0.0,0.0,5,1,63.97,0.76,0.0000,NaN
2016-10-01 02:00:00,2016-10-01 02:00:00-05:00,0.674067,0.0,0.0,0.0,5,1,62.23,0.79,0.0000,NaN
2016-10-01 03:00:00,2016-10-01 03:00:00-05:00,0.552900,0.0,0.0,0.0,5,1,61.33,0.81,0.0000,NaN
2016-10-01 04:00:00,2016-10-01 04:00:00-05:00,0.310983,0.0,0.0,0.0,5,1,61.13,0.82,0.0000,NaN
2016-10-01 05:00:00,2016-10-01 05:00:00-05:00,0.319317,0.0,0.0,0.0,5,1,60.43,0.83,0.0000,NaN
2016-10-01 06:00:00,2016-10-01 06:00:00-05:00,0.588933,0.0,0.0,0.0,5,1,60.02,0.84,0.0000,NaN
2016-10-01 07:00:00,2016-10-01 07:00:00-05:00,0.513133,0.0,0.0,0.0,5,1,59.91,0.84,0.0000,NaN
2016-10-01 08:00:00,2016-10-01 08:00:00-05:00,0.562567,0.0,0.0,0.0,5,1,61.76,0.82,0.0000,NaN
2016-10-01 09:00:00,2016-10-01 09:00:00-05:00,0.796333,0.0,0.0,0.0,5,1,66.10,0.75,0.0000,NaN


24 hour forecast based on 30 days

In [11]:
trace_1 = go.Scatter(
    x = df_exog_train.loc[start_train:end_train, 'Hourly_Usage'].index,
    y = df_exog_train.loc[start_train:end_train, 'Hourly_Usage'].values
)

In [12]:
trace_2 = go.Scatter(
    x = df_exog_train.loc[start_test:end_test, 'forecast'].index,
    y = df_exog_train.loc[start_test:end_test, 'forecast'].values
)

In [13]:
data = [trace_1, trace_2]

In [14]:
py.iplot(data, filename='sarimax_545')

------

Predict next month's energy usage

In [15]:
df_exog_month, arima_exog_results_month = fit_exog_arima(df, df_weather_austin, 545,
                                                         length=15, total_length=120, 
                                                         test_length=15*24)

In [16]:
mse_month, rmse_month, df_exog_train_month = plot_exog_arima(df, df_exog_month, 
                                                             arima_exog_results_month, 545,
                                                             length=15, total_length=120, 
                                                             test_length=15*24);

In [17]:
start_train_month, end_train_month, start_test_month, end_test_month = find_dates(545, length=15, 
                                                                                  total_length=120)

In [18]:
trace_1 = go.Scatter(
    x = df_exog_train_month.loc[start_train_month:end_test_month, 'Hourly_Usage'].index,
    y = df_exog_train_month.loc[start_train_month:end_test_month, 'Hourly_Usage'].values
)

In [19]:
trace_2 = go.Scatter(
    x = df_exog_train_month.loc[start_test_month:end_test_month, 'forecast'].index,
    y = df_exog_train_month.loc[start_test_month:end_test_month, 'forecast'].values
)

In [20]:
data = [trace_1, trace_2]

In [21]:
py.iplot(data, filename='sarimax_545_month')

Compare appliance usage to neighborhood

In [25]:
average_appliances = pickle.load(open('pickle_files/average_appliance.p', 'rb'))
df_545 = pickle.load(open('pickle_files/df_545.p', 'rb'))

In [27]:
appliances = ['air1','bedroom1','dishwasher1','dryg1','furnace1','livingroom1','microwave1','refrigerator1']

In [28]:
appliances_545 = monthly_usage(545,appliances)

In [92]:
total_appliances_545 = {}

In [ ]:
for appliance in appliances_545.keys():
    total_appliances_545[appliance]= sum(appliances_545[appliance].values)

In [75]:
total_appliances_545

{'air1': 467.25720000000013,
 'bedroom1': 16.787433333333354,
 'dishwasher1': 16.236383333333333,
 'dryg1': 7.8145166666666652,
 'furnace1': 175.0073666666662,
 'livingroom1': 68.521200000000093,
 'microwave1': 12.85586666666671,
 'refrigerator1': 41.574383333333344}

In [76]:
average_appliances

{'air1': 266.91244686692482,
 'bedroom1': 24.088478888888893,
 'dishwasher1': 11.447590041928724,
 'dryg1': 6.0303079710144951,
 'furnace1': 86.075987585034014,
 'livingroom1': 56.975688011695887,
 'microwave1': 7.6249509523809449,
 'refrigerator1': 64.59492322390571}

In [84]:
[x for x in total_appliances_545.values()]

[41.574383333333344,
 12.85586666666671,
 467.25720000000013,
 16.236383333333333,
 175.0073666666662,
 7.8145166666666652,
 68.521200000000093,
 16.787433333333354]

In [85]:
trace1 = go.Bar(
    x=[x for x in total_appliances_545.keys()],
    y=[y for y in total_appliances_545.values()],
    name='Your Home'
)

In [86]:
trace2 = go.Bar(
    x=[x for x in average_appliances.keys()],
    y=[y for y in average_appliances.values()],
    name='Homes in Your Neighborhood'
)

In [87]:
data = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)

In [88]:
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='appliances_comparison')

In [30]:
df_meta = pickle.load(open('pickle_files/df_meta_refined.p', 'rb'))

In [41]:
df_meta[['air1','']]

,dataid,active_record,building_type,program_579,program_baseline,program_energy_internet_demo,program_lg_appliance,program_verizon,program_ccet_group,program_civita_group,city,state,pv,date_enrolled,date_withdrawn,house_construction_year,total_square_footage,first_floor_square_footage,second_floor_square_footage,third_floor_square_footage,half_floor_square_footage,lower_level_square_footage,audit_2011,audit_2013_2014,survey_2011,survey_2012,survey_2013,survey_2014,number_of_nests,indoor_temp_min_time,indoor_temp_max_time,gas_ert_min_time,gas_ert_max_time,water_ert_min_time,water_ert_max_time,egauge_min_time,egauge_max_time,use,grid,gen,air1,air2,air3,airwindowunit1,aquarium1,bathroom1,bathroom2,bedroom1,bedroom2,bedroom3,bedroom4,bedroom5,car1,clotheswasher1,clotheswasher_dryg1,diningroom1,diningroom2,dishwasher1,disposal1,drye1,dryg1,freezer1,furnace1,furnace2,garage1,garage2,heater1,housefan1,icemaker1,jacuzzi1,kitchen1,kitchen2,kitchenapp1,kitchenapp2,lights_plugs1,lights_plugs2,lights_plugs3,lights_plugs4,lights_plugs5,lights_plugs6,livingroom1,livingroom2,microwave1,office1,outsidelights_plugs1,outsidelights_plugs2,oven1,oven2,pool1,pool2,poollight1,poolpump1,pump1,range1,refrigerator1,refrigerator2,security1,shed1,sprinkler1,utilityroom1,venthood1,waterheater1,waterheater2,winecooler1
dataid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8872,8872,None,Single-Family Home,None,yes,yes,None,None,CCET - Control,None,Austin,Texas,yes,2011-02-01,2013-06-10,2008.0,1422.0,1422.0,0.0,0.0,NaN,NaN,yes,,yes,yes,,,NaN,None,None,None,None,None,None,2012-05-02 01:00:00-05:00,2013-06-10 00:59:00-05:00,yes,yes,yes,yes,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,yes,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,yes,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1464,1464,None,Single-Family Home,None,yes,yes,None,None,CCET - Control,None,Austin,Texas,yes,2011-02-01,2014-03-06,2007.0,2345.0,1385.0,960.0,0.0,NaN,NaN,yes,,yes,yes,,,NaN,None,None,2012-04-13 09:26:26-05:00,2014-02-06 23:53:33-06:00,2012-05-01 13:00:00-05:00,2014-02-06 23:58:03-06:00,2012-10-19 19:00:00-05:00,2014-03-05 17:59:00-06:00,yes,yes,yes,yes,None,None,None,None,None,None,yes,yes,yes,None,None,None,yes,None,None,None,yes,yes,None,yes,None,None,None,None,None,None,None,None,None,None,None,yes,None,yes,None,None,None,None,None,None,None,yes,None,None,None,yes,None,None,None,None,None,None,yes,yes,None,yes,None,yes,None,None,None,None,None
8047,8047,None,Single-Family Home,None,yes,yes,None,None,CCET - Pricing Trial,None,Austin,Texas,yes,2011-02-01,2014-04-07,2009.0,3245.0,1779.0,1466.0,0.0,NaN,NaN,yes,,yes,,yes,,NaN,None,None,2012-04-11 18:59:52-05:00,2013-05-10 16:26:10-05:00,None,None,2013-04-16 00:00:00-05:00,2014-04-07 00:00:00-05:00,yes,yes,yes,yes,None,None,None,None,yes,None,None,None,None,None,None,None,yes,None,None,None,yes,yes,None,yes,yes,yes,None,None,None,None,None,None,None,None,None,yes,yes,None,None,None,None,None,None,None,None,yes,None,None,None,yes,None,None,None,None,None,None,None,yes,None,None,None,None,None,yes,None,None,None
8597,8597,None,Single-Family Home,None,yes,yes,None,None,CCET - UT Text,None,Austin,Texas,yes,2011-02-01,2014-10-31,2008.0,2122.0,1162.0,960.0,0.0,NaN,NaN,yes,,yes,,,,NaN,None,None,2012-04-11 18:59:53-05:00,2013-05-10 16:28:22-05:00,None,None,2012-10-26 19:00:00-05:00,2014-10-21 18:59:00-05:00,yes,yes,yes,yes,None,None,None,None,None,None,yes,None,None,None,None,None,yes,None,yes,None,yes,yes,yes,None,None,yes,None,None,None,None,None,None,None,None,None,yes,yes,yes,yes,None,None,None,None,None,None,None,None,None,None,yes,None,None,None,None,None,None,yes,yes,None,None,None,None,None,None,None,None,None
7982,7982,None,Single-Family Home,None,yes,yes,yes,None,CCET - Pricing Trial,None,Austin,Texas,yes,2011